<!-- same as self attention just mask half of the matrix -->
MaskedAttention(Q,K,V)=softmax(​QKT​/squt(dk)+M)V

In [21]:
import math
import torch
from torch import nn
import torch.nn.functional as F

In [22]:
class MaskAttention(nn.Module):
    def __init__(self,emb_dim):
        super().__init__()
        self.embdim = emb_dim
        self.k = nn.Linear(emb_dim,emb_dim)
        self.q = nn.Linear(emb_dim,emb_dim)
        self.v = nn.Linear(emb_dim,emb_dim)

    def forward(self,x):
        T, C = x.shape
        k = self.k(x)
        q = self.q(x)
        v = self.v(x)
        print(k.shape)
        attention = (q@k.transpose(-1,-2))/(self.embdim**0.5)
        mask = torch.tril(torch.ones(T, T, device=x.device))
        mask = mask.masked_fill(mask == 0, float('-inf'))
        mask = mask.masked_fill(mask == 1, 0.0)
        attention = attention + mask
        print(attention)
        attention = F.softmax(attention,dim=-1)
        out = attention @ v
        print(out)

        # result = v@F.Softmax(attention)

a = torch.randn(6, 5)
b = torch.tensor(a, requires_grad=True)
c1 = MaskAttention(emb_dim=b.shape[-1])
c1(b)

torch.Size([6, 5])
tensor([[-0.4029,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.0494,  0.0263,    -inf,    -inf,    -inf,    -inf],
        [ 0.1833,  0.4050, -0.1319,    -inf,    -inf,    -inf],
        [-0.2482, -0.3558, -0.1158, -0.4465,    -inf,    -inf],
        [-0.1443, -0.0889, -0.3397,  0.0169, -0.6495,    -inf],
        [ 0.2874,  0.3747, -0.1849,  0.5616, -0.6385, -0.6354]],
       grad_fn=<AddBackward0>)
tensor([[-0.5108,  0.3980,  0.1469,  0.6822,  0.3239],
        [-0.2769,  0.1798,  0.2572,  0.3694, -0.0795],
        [-0.2716,  0.1757,  0.0712,  0.2454, -0.1730],
        [-0.1953,  0.1891,  0.1777,  0.2617, -0.1189],
        [-0.2008,  0.2925,  0.2801,  0.3167, -0.0222],
        [-0.1014,  0.3059,  0.3471,  0.3449,  0.0103]], grad_fn=<MmBackward0>)


C:\Users\goura\AppData\Local\Temp\ipykernel_23256\3155872679.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.tensor(a, requires_grad=True)
